#### 1，朴素贝叶斯


#### 寻找文本的某些特征，然后根据这些特征将文本归为某个类。
像naive bayes这样的生成分类器构建了每个类的模型。通过观察，他们返回最有可能生成观察结果的类。

像logistic回归这样的区分分类器可以从输入中学习哪些特征对于区分不同的可能类最有用。

优点：在数据较少的情况下仍然有效，可以处理多类别问题。
缺点：对于输入数据的准备方式较为敏感。

In [1]:
%%html
<img src='1.png'>

#### 朴素贝叶斯的原理


#### 训练朴素贝叶斯的过程其实就是计算先验概率和似然函数的过程。

①先验概率P(c)的计算

②似然函数P(wi|c)的计算

由于是用词袋模型表示一篇文档d，对于文档d中的每个单词wi，
找到训练数据集中所有类别为c的文档，
数一数单词wi在这些文档（类别为c）中出现的次数：count(wi,c)

然后，再数一数训练数据集中类别为c的文档一共有多少个单词。计算二者之间的比值，
就是似然函数的值。似然函数计算公式如下：


其中V，就是词库。（有些单词在词库中，但是不属于类别C，那么 count(w,c)=0）

In [2]:
%%html
<img src='2.png'>

#### 利用朴素贝叶斯模型进行文本分类

#### 计算过程：

In [3]:
%%html
<img src='3.png'>

#### 1 载入数据集：6条文本及它们各自的类别，这6条文本作为训练集。

In [38]:
from numpy import *

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

#### 2 创建词汇表：利用集合结构内元素的唯一性，创建一个包含所有词汇的词表。

In [39]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

#### 3 把输入文本根据词表转化为计算机可处理的01向量形式：

In [43]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

#### 4 训练模型：在训练样本中计算先验概率 p(Ci) 和 条件概率 p(x,y | Ci)，本实例有0和1两个类别，所以返回p(x,y | 0)，p(x,y | 1)和p(Ci)。

In [44]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)      #change to ones() 
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:        
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)          #change to log()
    p0Vect = log(p0Num/p0Denom)          #change to log()

    return p0Vect,p1Vect,pAbusive

#### 5 分类：根据计算后，哪个类别的概率大，则属于哪个类别。



In [45]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

#### 6 测试程序

加载数据集+提炼词表；

训练模型：根据六条训练集计算先验概率和条件概率；

测试模型：对训练两条测试文本进行分类。

In [47]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print( testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print( testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

In [48]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


### 例子2

#### 1 对邮件的文本划分成词汇，长度小于2的默认为不是词汇，过滤掉即可。返回一串小写的拆分后的邮件信息。

In [49]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

#### 2 文档词袋模型：使用数组代替集合数据结构，可以保存词汇频率信息。



In [50]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index[word]] += 1
    return returnVec

#### 3 输入为25封正常邮件和25封垃圾邮件。50封邮件中随机选取10封作为测试样本，剩余40封作为训练样本。

　　　训练模型：40封训练样本，训练出先验概率和条件概率；

　　　测试模型：遍历10个测试样本，计算垃圾邮件分类的正确率。



In [53]:
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        # print wordList
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = range(50); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print( "classification error",docList[docIndex])
    print( 'the error rate is: ',float(errorCount)/len(testSet))
    #return vocabList,fullText

#### 2，SVM模型

In [ ]:
应用场景：文本分类、图像识别、主要二分类领域
SVM优点
1、解决小样本下机器学习问题。
2、解决非线性问题。
3、无局部极小值问题。（相对于神经网络等算法）
4、可以很好的处理高维数据集。
5、泛化能力比较强。

SVM缺点
1、对于核函数的高维映射解释力不强，尤其是径向基函数。
2、对缺失数据敏感

#### SVM的原理

#### 利用SVM模型进行文本分类

In [ ]:
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import jieba
from sklearn.externals import joblib
from sklearn.svm import SVC
import sys
 
 
def load_file_and_preprocessing():
    neg = pd.read_excel('data/neg.xls', header=None, index=None)
    pos = pd.read_excel('data/pos.xls', header=None, index=None)
    
    cw = lambda x: list(jieba.cut(x))
    pos['words'] = pos[0].apply(cw)
    neg['words'] = neg[0].apply(cw)
    
    # print pos['words']
    # use 1 for positive sentiment, 0 for negative
    y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))
    
    # 切割训练集和测试集
    x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos['words'], neg['words'])), y, test_size=0.2)
    
    np.save('data/y_train.npy', y_train)
    np.save('data/y_test.npy', y_test)
    return x_train, x_test
 
 
def build_sentence_vector(text, size, imdb_w2v):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec
 
 
# 计算词向量
def get_train_vecs(x_train, x_test):
    n_dim = 300
    # 初始化模型和词表
    imdb_w2v = Word2Vec(size=n_dim, min_count=10)
    imdb_w2v.build_vocab(x_train)
    
    # 在评论训练集上建模(可能会花费几分钟)
    imdb_w2v.train(x_train)
    
    train_vecs = np.concatenate([build_sentence_vector(z, n_dim, imdb_w2v) for z in x_train])
    # train_vecs = scale(train_vecs)
    
    np.save('data/train_vecs.npy', train_vecs)
    print(train_vecs.shape)
    # 在测试集上训练
    imdb_w2v.train(x_test)
    imdb_w2v.save('data/w2v_model.pkl')
    # Build test tweet vectors then scale
    test_vecs = np.concatenate([build_sentence_vector(z, n_dim, imdb_w2v) for z in x_test])
    # test_vecs = scale(test_vecs)
    np.save('data/test_vecs.npy', test_vecs)
    print(test_vecs.shape)
 
 
def get_data():
    train_vecs = np.load('data/train_vecs.npy')
    y_train = np.load('data/y_train.npy')
    test_vecs = np.load('data/test_vecs.npy')
    y_test = np.load('data/y_test.npy')
    return train_vecs, y_train, test_vecs, y_test
 
 
def svm_train(train_vecs, y_train, test_vecs, y_test):
    clf = SVC(kernel='rbf', verbose=True)
    clf.fit(train_vecs, y_train)
    joblib.dump(clf, 'data/svm_model.pkl')
    print(clf.score(test_vecs, y_test))
 
 
def get_predict_vecs(words):
    n_dim = 300
    imdb_w2v = Word2Vec.load('data/w2v_model.pkl')
    # imdb_w2v.train(words)
    train_vecs = build_sentence_vector(words, n_dim, imdb_w2v)
    # print train_vecs.shape
    return train_vecs
 
 
def svm_predict(string):
    words = jieba.lcut(string)
    words_vecs = get_predict_vecs(words)
    clf = joblib.load('data/svm_model.pkl')
    
    result = clf.predict(words_vecs)
    
    if int(result[0]) == 1:
        print(string, ' positive')
    else:
        print(string, ' negative')
 
 
if __name__ == '__main__':
    ##对输入句子情感进行判断
    string = '电池充完了电连手机都打不开.简直烂的要命.真是金玉其外,败絮其中!连5号电池都不如'
    # string='牛逼的手机，从3米高的地方摔下去都没坏，质量非常好'
    svm_predict(string)

#### 3，LDA主题模型

In [ ]:
LDA模型生成过程可描述如下：

    （１）文档ｄ中词项总数Ｎｄ服从泊松分布，其参数为ξ：Ｎｄ～Poisson（ξ）

    （２）对每篇文档ｄ∈｛１，２，．．．，｜Ｄ｜｝，按概率生成其主题分布：θ→ｄ～ Dirichlet（α→）；

    （３）对每个主题ｚ∈｛１，２，．．．，Ｋ｝，按概率生成其词项分布：φ→ｋ～ Dirichlet（β→）；

    （４）对文档ｄ中每个词wn的生成过程，其中ｎ∈｛１，２，．．．，Ｎｄ｝，有：         

        １）根据主题分布θ→ｄ生成文档ｄ词项ｗｎ主题：ｚｄ，ｎ～Multionmial（θ→ｄ）；         

        ２）根 据 词 项 分 布φｚｄ，ｎ→生 成 所 选 主 题 词 项：ｗｄ，ｎ～Multionmial（φ→ｚｄ，ｎ）。


#### pLSA、共轭先验分布

#### LDA

In [ ]:
其具体步骤总结如下：

输入：LDA模型语料库、KNN分类语料库

输出：待分类文本的分类结果

（１）通过文本语料库训练LDA模型并推断KNN训练和测试文本集的主题分布；

（２）选取特征词并修改主题分布；

（３）根据下式计算主题相似度

In [54]:
%%html
<img src='4.webp'>

#### 使用LDA生成主题特征，在之前特征的基础上加入主题特征进行文本分类

#### LDA数学八卦 lda2 合并特征